# Homework 2

## Question 1

Write a function that returns the name of a company for a given ticker symbol. 

You should scrape the data from Yahoo! Finance. [Here's an example page](https://finance.yahoo.com/quote/WMT/profile) using Walmart. Start by making your function modify the URL so it works for other ticker symbols. Then, look at the HTML source to find the pattern that you must search for to extract the company name. (Hint: it appears in the html in multiple places, some easier than others to extract).

I've started the function for you here. Modify/complete the function.

In [1]:
def unescape(s):
    s = s.replace('&#x27;', "'")
    s = s.replace('&amp', '&')
    return s

def get_name(ticker):
    import requests, re
    
    # put the URL here. Make sure it uses the ticker provided when the function is called 
    url = 'https://finance.yahoo.com/quote/{0}/profile'.format(ticker)
 
    
    # download
    req = requests.get(url)
    html = req.text

    
    try:
        # use a regular expression to find the name
        # (finish/modify the code)
        title_search = re.findall(r'<title>(.*?)</title>', html)
        str_title = title_search[0]
        str_title = unescape(str_title)
        stop_index = str_title.index('(')
        return str_title[0:stop_index]

    except:
#         if the regular expression fails, do nothing
        pass

Test your function using these tickers: `ADM`, `AMZN`, `MMM`, `M`, and `WMT`. Be sure it works as expected.

In [2]:
print(get_name('ADM'))
print(get_name('AMZN'))
print(get_name('MMM'))
print(get_name('M'))
print(get_name('WMT'))

Archer-Daniels-Midland Company 
Amazon.com, Inc. 
3M Company 
Macy's Inc 
Walmart Inc. 


## Question 2
Write a function that uses the same web page of data to find the address of the company's headquarters.

For example, your function should work as follows:

    In  []: get_address('TSLA')
    Out []: '3500 Deer Creek Road, Palo Alto, CA, 94304'
    
Be sure to test your function with some other stock tickers.

Hint: You'll probably want to capture different parts of the address (street address, city, state, zip) separately and then join them together into a string separated by commas before returning a value. Search through the html to see where the different parts appear.

In [3]:
def get_address(ticker):
    import requests, re
    
    # download file using the url for the right ticker (like in Q2)
    url = 'https://finance.yahoo.com/quote/{0}/profile?p={0}'.format(ticker, ticker)

 
    # download
    req = requests.get(url)
    html = req.text
    
    try:
        # here you should be using a regular expression like in Q2, but try to capture the individual address components.
        # combine them into a string and return the string
        street_add= re.findall(r'<!-- react-text: 9 -->(.*?)<', html)[0]
        city_state_zip = re.findall(r'<!-- react-text: 11 -->(.*?)<', html)[0]
        return unescape((street_add + ', ' + city_state_zip))
    except:
        pass

In [4]:
get_address('AMZN')

'410 Terry Avenue North, Seattle, WA 98109'

In [5]:
get_address('F')

'One American Road, Dearborn, MI 48126'

In [6]:
get_address('WHR')

'2000 North M-63, Benton Harbor, MI 49022'

## Question 3

Now I want you to write a *geolocating* function that takes a ticker symbol and returns the latitude and longitude of the firm's headquarters. To do this, we will use Mapquest's API (<b>A</b>pplication <b>P</b>rogramming <b>I</b>nterface), which is a web-based system for providing raw data rather than HTML pages.

Before accessing the API you need to create an account which generates an API Key. The account is free and allows you to make a limited number of queries. To sign up, go [here](https://developer.mapquest.com/user/register) and choose Sign Up. You are assigned a key when you register. It will look something like this: 

<pre>zAWt8Aye6YvhIupZMzctaAfkSkEnaccN</pre>

Once you have created your key, you can query the API using a URL like this:

http://www.mapquestapi.com/geocoding/v1/address?key=API_KEY&location=1309+E+10th+Street%2C+Bloomington%2C+IN%2C+47401

(This link won't work until you replace `API_KEY` with your personal key.)

Notice there is a special formatting to deal with spaces and other punctuation. An easy way to request a web site like this is as follows:

In [179]:
import requests
API_KEY='7eA2xBsCAHvN9XAlODv9Jkw8EqDXTqho'     # make sure to replace this with your own key
url = 'http://www.mapquestapi.com/geocoding/v1/address'
params = {'key': API_KEY, 'location': '1309 E 10th Street, Bloomington, IN, 47401'}
requests.get(url, params=params).text

'{"info":{"statuscode":0,"copyright":{"text":"\\u00A9 2020 MapQuest, Inc.","imageUrl":"http://api.mqcdn.com/res/mqlogo.gif","imageAltText":"\\u00A9 2020 MapQuest, Inc."},"messages":[]},"options":{"maxResults":-1,"thumbMaps":true,"ignoreLatLngInput":false},"results":[{"providedLocation":{"location":"1309 E 10th Street, Bloomington, IN, 47401"},"locations":[{"street":"1309 E 10th St","adminArea6":"","adminArea6Type":"Neighborhood","adminArea5":"Bloomington","adminArea5Type":"City","adminArea4":"Monroe","adminArea4Type":"County","adminArea3":"IN","adminArea3Type":"State","adminArea1":"US","adminArea1Type":"Country","postalCode":"47405-1701","geocodeQualityCode":"L1AAB","geocodeQuality":"ADDRESS","dragPoint":false,"sideOfStreet":"L","linkId":"rnr3442633|i21368598","unknownInput":"","type":"s","latLng":{"lat":39.171598,"lng":-86.517943},"displayLatLng":{"lat":39.171798,"lng":-86.517941},"mapUrl":"http://www.mapquestapi.com/staticmap/v5/map?key=7eA2xBsCAHvN9XAlODv9Jkw8EqDXTqho&type=map&size=

The response from the server is data from the database stored in JSON format. But you can ignore that it's JSON and just use a regular expression to find the pattern you're looking for.

As an example, your code should return a tuple *exactly* like this:

    In  [2]: geolocate('TSLA')
    Out [2]: (37.394738, -122.149112, '3500 Deer Creek Road, Palo Alto, CA, 94304')
    
That is, it returns a tuple with the latitude, longitude, and address.

Use the template below to write the function. The function you write for this question should start by calling the function you wrote in Q2 to get the address. You can then use this with the API to get the location

In [7]:
API_KEY = '7eA2xBsCAHvN9XAlODv9Jkw8EqDXTqho'  # replace this with your own key

In [8]:
def geolocate(ticker):
    import requests, re
    
    addr = get_address(ticker)
    
    url = 'http://www.mapquestapi.com/geocoding/v1/address'
    params = {'key': API_KEY, 'location': addr}      # need to include address here
    html = requests.get(url, params=params).text

    try:
                                # extract the latitude and longitude from the webpage using a regular expression
        lat = float(re.findall(r'"lat":(.*?),', html)[0])
        lng = float(re.findall(r'"lng":(.*?)}', html)[0])
                        
        return(lat, lng, addr)
    except:
        print('Unable to find coordinates')

In [10]:
geolocate('TSLA')

(37.394802, -122.149157, '3500 Deer Creek Road, Palo Alto, CA 94304')

## Question 4

Now let's say we want to calculate the distance between two corporate headquarters.

Given two pairs of latitude and longitude, $(\varphi_1, \lambda_1)$ and $(\varphi_2, \lambda_2)$, we can calculate the distance between them using the [Haversine formula](https://en.wikipedia.org/wiki/Haversine_formula#The_haversine_formula):

$${\displaystyle d=2r\arcsin \left({\sqrt {\operatorname {hav} (\varphi _{2}-\varphi _{1})+\cos(\varphi _{1})\cos(\varphi _{2})\operatorname {hav} (\lambda _{2}-\lambda _{1})}}\right)}$$

where $d$ is the distance, $\operatorname{hav}$ is the Haversine function, $$\operatorname {hav} (\theta )=\sin ^{2}\left({\frac {\theta }{2}}\right),$$ and $r$ is the radius of the earth ([3958.7613 miles](https://en.wikipedia.org/wiki/Earth_radius#Mean_radius)). Note that this formula requires that the latitude and longitude are given in *radians*, not the degrees that Mapquest gives.

Use the template below to write a function that takes two ticker symbols and returns the distance between the headquarters of the two firms. Note that the python function for $\arcsin$ is `asin`.

In [11]:
def haversine(theta):
    '''Haversine function'''
    from math import sin
    return  (sin(theta/2)**2)                 # enter the haversine function (from above) return the value for a given theta

def firm_dist(ticker1, ticker2):
    '''Returns the distance in miles between two firm HQs'''
    
    from math import radians, cos, sin, asin, sqrt
    
    # get lat/lng of two tickers by calling function in Q3
    # We're using the underscore because the function we wrote in Q3 returns three values
    # and the underscore just means discard that value (we don't need it)
    lat1, lng1, _  = geolocate(ticker1)
    lat2, lng2, _  = geolocate(ticker2)
        
    # change degrees to radians using the radians function we imported
    # hint: use the map function
    lat1, lng1, lat2, lng2 = map( radians, (lat1, lng1, lat2, lng2))

    r = 3958.7613
    
    return  2 * r * asin( sqrt(haversine(lat2-lat1)+(cos(lat1)*cos(lat2)*haversine(lng2-lng1))))    # I've started the distance function for you.  Finish it

In [12]:
d = firm_dist('GOOG', 'FB')

print(f'Google and Facebook are {d:.1f} miles apart')

Google and Facebook are 5.4 miles apart
